In [1]:
import pandas as pd
import numpy as np
import logging
from smart_open import smart_open
import requests as req
import re
import os
import tarfile

In [5]:
# get current directory to save it to a string with info about our local directory
path=os.getcwd()
path

'/media/jlealtru/data_files/github/nlp_experiments'

In [6]:
os.path.isdir(path+'/aclImdb')

False

In [13]:
# We will check if the data already exist in our local enviroment. If not we will download it from the Standford
# webpage

if os.path.isdir(path+'/aclImdb'):
    print('Files available for work')
else:
    print("Downloading IMDB archive...")
    url = u'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
    r = req.get(url)
    with smart_open('standford', 'wb') as f:
        f.write(r.content)
    print('file download complete')
    # if error here, try `tar xfz aclImdb_v1.tar.gz` outside notebook, then re-run this cell
    tar = tarfile.open('standford', mode='r')
    tar.extractall()
    tar.close()
    print('extraction complete, files available for work')


file download complete
extraction complete, files available for work


In [16]:
# We now need to parse the files we just downloaded into our machine. We define a function that
# looks into a folder and parses the contents into a list. We also need to define the sentiment for
# the classification.
def get_review_content(path_to_folder, sentiment):
    contents=[]
    for filename in os.listdir(path_to_folder+sentiment):
    #print (filename)
    #print (os.getcwd())
        with open(path_to_folder+sentiment+'/'+filename, 'rb') as openfile:
        #openfile.read()
            content=openfile.read()
            contents.append(content)
    return contents


In [17]:
# start with the negative freviews
contents_negative_train=get_review_content(path+'/aclImdb/train/', 'neg')
contents_negative_test=get_review_content(path+'/aclImdb/test/', 'neg')
len(contents_negative_test+contents_negative_train)

25000

In [19]:
# we repeat with the positive reviews
contents_positive_train=get_review_content(path+'/aclImdb/train/', 'pos')
contents_positive_test=get_review_content(path+'/aclImdb/test/', 'pos')
len(contents_positive_test+contents_positive_train)

25000

In [28]:
# we now parse the text with undefined 
contents_undefined=get_review_content(path+'/aclImdb/train/', 'unsup')
len(contents_undefined)

50000

In [31]:
#clean the text
#clean the text
import multiprocessing
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import stem_text
from gensim.parsing.preprocessing import strip_numeric
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import strip_non_alphanum

# we can do this whole thing across a number of processors 
# initialize pool of processes
p = multiprocessing.cpu_count()

def clean_text(text):
    # filter ords greater than 120
    #wl = filter(lambda x: ord(x) < 128, text)

    # pass through pre-processing filter
    wl = preprocess_string(text, filters = [stem_text, strip_numeric, 
                                          strip_punctuation, remove_stopwords, 
                                          strip_short, strip_non_alphanum])
    return wl
                    
    #clean_text = p.map(clean_text, labeled['review'].values.tolist())

In [37]:
# now define a function to clean the text data that we can apply to all of our functions
import gensim
assert gensim.models.doc2vec.FAST_VERSION> -1

In [ ]:
with multiprocessing.Pool(processes=8) as pool:
    clean=pool.map(clean_text, contents_undefined)
    
x

In [26]:

#we now create a dataframe with the reviews and the sentiment.
imdb_train=pd.DataFrame({'reviews':contents_positive_train+contents_undefined})
len(imdb_train)
#imdb_train.head()


62500